In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.2"

In [ ]:
# 나눔 폰트 설치

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
# 데이터분석
import pandas as pd
import numpy as np

# 자연어처리 
import konlpy
from nltk import Text
from konlpy.tag import Okt 

# 한글폰트 사용 in colab
%matplotlib inline  
import matplotlib.pyplot as plt
import matplotlib as mpl 
import matplotlib.font_manager as fm  


path = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/MyDrive/Colab Notebooks/dataset/train.csv'

#pandas read_csv로 불러오기
data = pd.read_csv(filename)
data.head()

In [ ]:
#mapping labels
labelsname = '/content/drive/MyDrive/Colab Notebooks/dataset/labels_mapping.csv'

#pandas read_csv로 불러오기
labels = pd.read_csv(labelsname)
labels['소분류'].unique()

In [ ]:
len(data)

In [ ]:
data['label'].isnull().sum()
# 결측치 확인
# 결측치가 없다고 나옴
# But NaN이 0이므로 0값을 찾아야 함

In [ ]:
num0 = data.loc[data['label'] == 0]
len(num0)
# 결측치 갯수 142571개

In [ ]:
data['label'].unique()
# 없는 label이 있음

## 데이터 탐색

In [ ]:
# import pandas_profiling

In [ ]:
# pr=data.profile_report() # 프로파일링 결과 리포트를 pr에 저장
# pr

In [ ]:
# --> 122         category_alias_name = category_alias_name.replace("_", " ")
# AttributeError: 'NoneType' object has no attribute 'replace' 오류로 실행 x, null값 해결할 것 
# pr.to_file('./pr_report.html') # pr_report.html 파일로 저장

In [ ]:
data.info()

In [ ]:
# 라벨순대로 정렬할 것
data = data.sort_values(by=['label'], axis=0)
data.head()

In [ ]:
import re
tokenizer = Okt()

## 특수문자 조회, 반환

In [ ]:
# 특수문자 제거함수 예제
a = "you,are○my:sun"

In [ ]:
new_a = ''.join(filter(str.isalnum, a))
new_a

In [ ]:
# 특수문자만 제거하는 함수

def remove_specialCha(text_list):
    txt_list=[]
    for text in text_list:
        txt = re.sub('[^가-힣a-z]', ' ', str(text)) #한글과 영어 소문자만 남기고 다른 글자 모두 제거
        txt_list.append(txt)
    return txt_list

In [ ]:
# example = ['본 연구에서는 현화식물내 화분의 발달 과정의 특징과 최종 산물인 화분의 형태학적 형','ㅇ 고숙련 퇴직인력 활용을 통해 생산공정 분야 전문인력 양성\n\n  - 전체 사업']

In [ ]:
# remove_specialCha(example)

In [ ]:
# 데이터프레임 칼럼을 리스트로 변환
# dataframe->ndarray로 변환
array_df = data['요약문_연구목표'].values
# ndarray->list로 변환
list_df = array_df.tolist()
list_df

In [ ]:
SummaryOfResearch = remove_specialCha(list_df)

In [ ]:
len(SummaryOfResearch)

In [ ]:
# SummaryOfResearch1000 = SummaryOfResearch[:1000]
SummaryOfResearch10 = SummaryOfResearch[:10]
SummaryOfResearch10

In [ ]:
# 코드 출력 시간
import time 
time0 = time.time()

## 형태소 분석기

In [ ]:
def text_preprocessing(text_list):
    
    stopwords = ['을', '를', '이', '의', '가', '은', '는','으로', '에', '에서','에서는', '인', '적', '간'] #불용어 설정
    tokenizer = Okt() #형태소 분석기 생성
    
    for i, j in enumerate(text_list):
        token_list=[]
        for word in tokenizer.morphs(j):#형태소 분석
            if word not in stopwords:#형태소 분석 결과 중 stopwords에 해당하지 않는 것만 추출   
                token_list.append(word)
        text_list[i] = ' '.join(token_list)
        return token_list

    # for text in text_list:
    #     token = tokenizer.morphs(text) #형태소 분석
    #     token2 = t for t in token if t not in stopwords 
    #     print(token2) 
    #     token_list.append(token2)
    # return token_list

In [ ]:
text_preprocessing(SummaryOfResearch1000)
# print(time.time()-time0)#1000개에 3분 33초

In [ ]:
# labels가 1인 값 조회, 원자력
num1 = data.loc[data['label'] == 1]
num1

In [ ]:
len(num1)

In [ ]:
# labels가 2인 값 조회, 핵융합
num2 = data.loc[data['label'] == 2]
num2

In [ ]:
len(num2)

In [ ]:
# 모든 칼럼 별 확인
# 사업명
# 사업_부처명
# 계속과제여부
# 내역사업명	
# 과제명
# 요약문_연구목표	
# 요약문_연구내용	
# 요약문_기대효과
# 요약문_한글키워드
# 요약문_영문키워드->nltk로 빈도수 분석

### 공통적으로 들어가는 명사 확인

In [ ]:
from konlpy.tag import Okt 
okt = Okt()

In [ ]:
num1['사업명']

In [ ]:
num2['사업명']

In [ ]:
# 각 칼럼별 텍스트 for문으로 추출함

In [ ]:
num1_buisnessName = []
for i in num1['사업명']:
    num1_buisnessName.append(i)
print(num1_buisnessName)

In [ ]:
num1_buisnessName_result = ' '.join(num1_buisnessName)
print(num1_buisnessName_result)

In [ ]:
type(num1_buisnessName_result)

In [ ]:
num2_buisnessName = []
for i in num2['사업명']:
    num2_buisnessName.append(i)
print(num2_buisnessName)

In [ ]:
num2_buisnessName_result = ' '.join(num2_buisnessName)
print(num2_buisnessName_result)

In [ ]:
# 명사 추출
num1_buisnessName_result_noun = okt.nouns(num1_buisnessName_result)
num1_buisnessName_result_noun

In [ ]:
num2_buisnessName_result_noun = okt.nouns(num2_buisnessName_result)
num2_buisnessName_result_noun

In [ ]:
# ValueError: Iterable over raw text documents expected, string object received. 
# 오류 날 시 string에 list 표시하여 바꾸기(list만 받아들임)

# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(num1_buisnessName_result_noun)
vect.vocabulary_

In [ ]:
vect = CountVectorizer()
vect.fit(num2_buisnessName_result_noun)
vect.vocabulary_

In [ ]:
# 자연어처리 목적 : 공통적으로 원자력과 관련 있는 단어 추출
# 해당 단어가 포함된 값은 label을 '01. 원자력 발전'라고 함, 02. 핵융합발전과 겹칠 수 있으니 어떻게 구별할지 유의
# bow인코딩벡터로 변환하여 해당 값이 있는 칼럼만 선정해서 훈련??
# 토큰이 나타난 횟수를 기준으로 단어장 구성 가능 

# '및핵': 26,
# '방사선': 27,
# '방폐물': 28,
# '원자력': 57,
# '융합': 59,
# '핵비': 84,

# 해결 -> 형태소 분석기로 다시 쪼개기, TF-IDF 방식으로 빈도수에 따른 중요도로 1,2 카테고리 분류

In [ ]:
# num1사업명 칼럼을 Bow인코딩 벡터로 변환
# 앞의 토큰 리스트 순서대로 없는 토큰은 0, 있는 토큰은 1로 나타냄 
vect.transform(num1['사업명']).toarray()

# 타 칼럼도 똑같이 분석 후 전체 데이터셋에 적용 
# 관련 있는 단어면 칼럼이 달라도 똑같이 적용하면 될 지??

## 형태소 분석